In [1]:
%load_ext autoreload
%autoreload 2

### Load Lib

In [3]:
import util as ut
import re
import os
from tqdm import tqdm
import yaml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [166]:
sql_file_name = "job_all.sql"
tab_dot_col_list = ['mc.note', 't.title', 'n.name', 'lt.link']
conn, cur = ut.get_db_connecter()
tables = ut.get_table(cur)
schema = ut.get_all_schema(cur)
short_cut_dict = ut.get_short_cut_dict(cur)
sql_queries = ut.read_strings(sql_file_name)

In [240]:
short_cut_dict

{'an': 'aka_name',
 'at': 'aka_title',
 'ci': 'cast_info',
 'chn': 'char_name',
 'cct': 'comp_cast_type',
 'cn': 'company_name',
 'ct': 'company_type',
 'cc': 'complete_cast',
 'it': 'info_type',
 'k': 'keyword',
 'kt': 'kind_type',
 'lt': 'link_type',
 'mc': 'movie_companies',
 'mi': 'movie_info',
 'mii': 'movie_info_idx',
 'mk': 'movie_keyword',
 'ml': 'movie_link',
 'n': 'name',
 'pi': 'person_info',
 'rt': 'role_type',
 't': 'title'}

In [167]:
attached_sql_queries_all = ut.attach_con_sels(cur, sql_queries)

100%|██████████| 113/113 [00:47<00:00,  2.38it/s]


In [184]:
attached_sql_queries = ut.filter_attached_sql_with_like_columns(attached_sql_queries_all, tab_dot_col_list)
sql_sets = set([x[0] for x in attached_sql_queries])

In [239]:
with open("job_LPLM.sql", 'w') as f:
    for sql_query in sorted(sql_sets):
        f.write(sql_query + "\n")

In [185]:
cond_pred_dict_all = ut.get_cond_pred_dict_from_attached_sql_queries(attached_sql_queries)
cond_pred_dict = ut.get_cond_pred_dict_from_attached_sql_queries(attached_sql_queries, tab_dot_col_list)

100%|██████████| 59/59 [00:00<00:00, 9206.25it/s]


In [186]:
cond_pred_dict_all

{"ct.kind = 'production companies'": 0.005,
 "it.info = 'bottom 10 rank'": 0.008929,
 "mc.note !~~ '%(as Metro-Goldwyn-Mayer Pictures)%'": 0.514131,
 't.production_year > 2000': 0.551089,
 "mc.note !~~ '%(TV)%'": 0.358804,
 "mc.note ~~ '%(USA)%'": 0.194591,
 't.production_year > 1990': 0.694659,
 "cn.country_code <> '[pl]'": 0.893967,
 "cn.name ~~ '%Film%'": 0.181588,
 "cn.name ~~ '%Warner%'": 0.010255,
 "k.keyword = 'sequel'": 7e-06,
 "lt.link ~~ '%follows%'": 0.0001,
 "t.title ~~ '%Money%'": 9.3e-05,
 't.production_year = 1998': 0.018933,
 "mc.note ~~ '%(theatrical)%'": 0.107915,
 "mc.note ~~ '%(France)%'": 0.011542,
 't.production_year > 2005': 0.404304,
 "k.keyword = 'character-name-in-title'": 7e-06,
 "n.name ~~ '%Bert%'": 0.0001,
 "cn1.country_code = '[nl]'": 0.010233,
 "it1.info = 'rating'": 0.008929,
 "it2.info = 'rating'": 0.008929,
 "kt1.kind = 'tv series'": 0.005,
 "kt2.kind = 'tv series'": 0.005,
 "lt.link ~~ '%follow%'": 0.00032,
 "mi_idx2.info < '3.0'": 0.481213,
 't2.pro

In [187]:
len(cond_pred_dict)

53

In [211]:
cond_list = list(cond_pred_dict)
table_col_dict = ut.get_table_col_dict_from_cond_list(cond_list, cur)

In [212]:
table_col_dict

{('mc', 'note'): ['%(1994)%',
  '%(200%)%',
  '%(2006)%',
  '%(2007)%',
  '%(Blu-ray)%',
  '%(France)%',
  '%(Japan)%',
  '%(TV)%',
  '%(USA)%',
  '%(VHS)%',
  '%(as Metro-Goldwyn-Mayer Pictures)%',
  '%(co-production)%',
  '%(presents)%',
  '%(theatrical)%',
  '%(worldwide)%'],
 ('lt', 'link'): ['%follow%', '%follows%', 'features'],
 ('t', 'title'): ['%Champion%',
  '%Freddy%',
  '%Jason%',
  '%Kung%Fu%Panda%',
  '%Loser%',
  '%Money%',
  '%Mord%',
  '%Movie%',
  '%Murder%',
  '%murder%',
  'Birdemic%',
  'Champion%',
  'Dragon Ball Z%',
  'Kung Fu Panda%',
  'Loser%',
  'One Piece%',
  'Saw%',
  'Shrek 2',
  'Vampire%'],
 ('n', 'name'): ['%An%',
  '%Ang%',
  '%Angel%',
  '%B%',
  '%Bert%',
  '%Downey%Robert%',
  '%Tim%',
  '%Yo%',
  '%Yu%',
  'A%',
  'B%',
  'X%',
  'Z%']}

### Count rows (all, not null)

In [ ]:
for table in ['link_type', 'movie_companies', 'name', 'title']:
    print(table, ut.get_count_data_strings_from_table(table, cur))

for table, col in [('link_type', 'link'), ('movie_companies', 'note'), ('name', 'name'), ('title', 'title')]:
    print(table, ut.get_count_data_strings_from_table_col(table, col, cur))

link_type 17
movie_companies 2609128
name 4167490
title 2528311
link_type 17
movie_companies 1337139
name 4167490
title 2528311


### Collect columns and predicates

In [199]:
table_col_dict

for table_col, test_queries in table_col_dict.items():
    data_strings = ut.get_data_strings_from_table_col(table_col, cur)
    print(table_col, data_strings[:2])
    table, col = table_col
    data_name = f"{table}.{col}"
    data_filepath = f"data/{data_name}/{data_name}.txt"
    test_filepath = f"data/{data_name}/query/CEB/test.txt"
    os.makedirs(os.path.dirname(data_filepath), exist_ok=True)
    os.makedirs(os.path.dirname(test_filepath), exist_ok=True)

    print(f"{data_filepath = }")
    with open(data_filepath, 'w') as f:
        for data_string in data_strings:
            if data_string is None:
                data_string = ''
            f.write(data_string + "\n")

    print(f"{test_filepath = }")
    with open(test_filepath, 'w') as f:
        for test_query in test_queries:
            f.write(test_query + "\n")
    

('movie_companies', 'note') ['(2006) (worldwide) (TV)', '(2012) (worldwide) (all media)']
data_filepath = 'data/movie_companies.note/movie_companies.note.txt'
test_filepath = 'data/movie_companies.note/query/CEB/test.txt'
('link_type', 'link') ['followed by', 'remake of']
data_filepath = 'data/link_type.link/link_type.link.txt'
test_filepath = 'data/link_type.link/query/CEB/test.txt'
('title', 'title') ["Josie Duggar's 1st Shoes", '(#2.8)']
data_filepath = 'data/title.title/title.title.txt'
test_filepath = 'data/title.title/query/CEB/test.txt'
('name', 'name') ['A., David', '-Alverio, Esteban Rodriguez']
data_filepath = 'data/name.name/name.name.txt'
test_filepath = 'data/name.name/query/CEB/test.txt'


### Make pattern

In [200]:
def token_type(x):
    if '%' in x or '_' in  x:
        return x
    else:
        return 'w'

def get_pattern_len_info(query):
    parsed = ut.parse_like_query(query)
    output = []
    for token in parsed:
        if '%' not in token and '_' not in token:
            output.append(len(token))
    return output

def make_pattern(query):
    parsed = ut.parse_like_query(query)
    # pattern_tokens = ['w' if '_' not in x and '%' not in x else x for x in parsed]
    pattern_tokens = [token_type(x) for x in parsed]
    pattern = ''.join(pattern_tokens)

    return pattern

def get_pattern_info(queries):
    pattern_dict = {}
    for query in queries:
        pattern = make_pattern(query)
        pattern_len_info = get_pattern_len_info(query)
        if pattern not in pattern_dict:
            pattern_dict[pattern] = set()
        pattern_dict[pattern].add(tuple(pattern_len_info))
    
    # pattern_dict = {x: [list(t) for t in y] for x, y in pattern_dict.items()}
    pattern_dict = {x: list(sorted(y, key=lambda x: x[-1])) for x, y in pattern_dict.items()}
    pattern_dict = {x: list(sorted(y, key=lambda x: x[0])) for x, y in pattern_dict.items()}
    # pattern_dict = {x: list(zip(*y)) for x, y in pattern_dict.items()}
    return pattern_dict
    

In [201]:
column_names = sorted(os.listdir('data/'))
for column_name in column_names:
    print(column_name)
    query_path = f'data/{column_name}/query/CEB/test.txt'
    queries = ut.read_strings(query_path)
    pattern_dict = get_pattern_info(queries)
    workload_path = f'data/{column_name}/query/CEB/workload.yml'
    print(pattern_dict)
    with open(workload_path, 'w') as f:
        yaml.dump(pattern_dict, f, default_flow_style=False)
    # for query in queries:
    #     pattern = make_pattern(query)
    
    # print(queries[0])
    # print(ut.parse_like_query(queries[0]))
    # print(make_pattern(queries[0]))
    


link_type.link
{'%w%': [(6,), (7,)], 'w': [(8,)]}
movie_companies.note
{'%w%': [(4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (15,), (33,)], '%w%w%': [(4, 1)]}
name.name
{'%w%': [(1,), (2,), (3,), (4,), (5,)], '%w%w%': [(6, 6)], 'w%': [(1,)]}
title.title
{'%w%': [(4,), (5,), (6,), (8,)], '%w%w%w%': [(4, 2, 5)], 'w%': [(3,), (5,), (7,), (8,), (9,), (13,)], 'w': [(7,)]}


### Gen train data from pattern

In [233]:
# def gen_query_a(data_string, pattern, pattern_len_info):
#     pass

def gen_query(db, pattern, pattern_len_info, n_repeat=1):
    def get_range(len_ranges, w_id, remain_range=[1, 1e6], remain_min_lens=[]):
        allowed_len = remain_range[1] - remain_range[0] + 1 - sum(remain_min_lens)
        if allowed_len <= 0:
            print(f"{remain_range = }")
            print(f"{remain_min_lens = }")
            print(f"{len_ranges = }")
            print(f"{w_id = }")
        assert allowed_len > 0, allowed_len
        min_len = min(len_ranges[w_id][0], allowed_len)
        max_len = min(len_ranges[w_id][1], allowed_len)
        return min_len, max_len

    parsed = ut.parse_like_query(pattern)
    n_words = parsed.count('w')
    if parsed[0] == '%' and parsed[-1] == '%':
        query_type = 'substring'
    elif parsed[-1] == '%':
        query_type = 'prefix'
    elif parsed[0] == '%':
        query_type = 'suffix'
    elif n_words == 1:
        query_type = 'exact'
    

    output = set()
    len_ranges = [[min(x), max(max(x), min(x) + 1)] for x in list(zip(*pattern_len_info))]
    np.random.seed(0)
    if query_type == 'exact':
        assert n_words == 1
        min_len, max_len = len_ranges[0]
        for data_string in db:
            if '_' in data_string or '%' in data_string:
                continue
            if len(data_string) <= max_len + 2:
                output.add(data_string)
        output = list(output)
        return output

    if n_repeat > 1:
        db = db * n_repeat
        
    for data_string in tqdm(db):
        if '_' in data_string or '%' in data_string:
            continue
        predicate = None

        remain_range = [1, len(data_string)]
        tokens = []

        # get all min_len
        min_lens = []
        for w_id in range(n_words):
            min_len, max_len = get_range(len_ranges, w_id)
            min_lens.append(min_len)
        if sum(min_lens) > len(data_string):
            continue
        # print("start query gen")
        # print(f"{min_lens = }")
        # print(f"{len(data_string) = }")

        if query_type == 'suffix':
            w_id = n_words - 1
            min_lens.pop()
            min_len, max_len = get_range(len_ranges, w_id, remain_range, min_lens)
            length = np.random.randint(min_len, max_len+1)
            last_token = data_string[-length:]
            remain_range = [remain_range[0], remain_range[1] - len(last_token)]
        
        
        for w_id in range(n_words):
            if len(min_lens) > 0:
                min_lens.pop(0)

            min_len, max_len = get_range(len_ranges, w_id, remain_range, min_lens)
            if query_type == 'prefix' and w_id == 0:
                length = np.random.randint(min_len, max_len+1)
                token = data_string[:length]
                remain_range = [remain_range[0] + len(token), remain_range[1]]
            elif query_type == 'suffix' and w_id == range(n_words) - 1:
                token = last_token
            else: # substring
                length = np.random.randint(min_len, max_len+1)
                # print(f"{n_words = }")
                # print(f"{length = }")
                # print(f"{remain_range = }")
                # print(f"{min_lens = }")
                # print(f"{sum(min_lens) = }")
                # print(f"{remain_range[0], (remain_range[1] + 1) - length + 1 - sum(min_lens) = }")
                start_pos = np.random.randint(remain_range[0], (remain_range[1] + 1) - length + 1 - sum(min_lens))
                start_idx = start_pos - 1
                token = data_string[start_idx: start_idx+length]
                remain_range = [start_pos + length, remain_range[1]]
        
            tokens.append(token)
                
        # tokens = [re.sub('%|_', '', token) for token in tokens]
        predicate = '%'.join(tokens)
        if query_type == 'prefix':
            predicate = predicate + "%"
        elif query_type == 'suffix':
            predicate = "%" + predicate
        else:
            predicate = "%" + predicate + "%"
            
        output.add(predicate)
    output = list(output)

    return output

In [234]:
column_names = sorted(os.listdir('data/'))
# column_names = ["movie_companies.note", "title.title", "name.name", "link_type.link"]
# column_names = ["title.title"]
column_names = ["title.title"]
for column_name in column_names:
    print(column_name)
    data_path = f'data/{column_name}/{column_name}.txt'
    db = ut.read_strings(data_path)
    workload_path = f'data/{column_name}/query/CEB/workload.yml'
    test_path = f'data/{column_name}/query/CEB/test.txt'
    valid_path = f'data/{column_name}/query/CEB/valid.txt'
    train_path = f'data/{column_name}/query/CEB/train.txt'
    test_queries = ut.read_strings(test_path)
    n_repeat = 1000 if column_name == 'link_type.link' else 1
    with open(workload_path) as f:
        workload = yaml.load(f, yaml.FullLoader)
    print(workload)
    # print(pattern_dict)
    # with open(workload_path, 'w') as f:
    #     yaml.dump(pattern_dict, f, default_flow_style=False)

    p_valid = 0.1
    
    train_queries = []
    valid_queries = []
    for pattern, pattern_len_info in workload.items():
        generated_queries = gen_query(db, pattern, pattern_len_info, n_repeat)
        generated_queries = list(set(generated_queries) - set(test_queries))
        train_queries_part, valid_queries_part = train_test_split(generated_queries, test_size=p_valid, random_state=0)
        train_queries.extend(train_queries_part)
        valid_queries.extend(valid_queries_part)
    
    train_queries = sorted(set(train_queries))
    valid_queries = sorted(valid_queries)
    
    with open(train_path, 'w') as f:
        f.writelines('\n'.join(train_queries))
        
    with open(valid_path, 'w') as f:
        f.writelines('\n'.join(valid_queries))
    

    # for query in queries:
    #     pattern = make_pattern(query)
    
    # print(queries[0])
    # print(ut.parse_like_query(queries[0]))
    # print(make_pattern(queries[0]))
    


title.title
{'%w%': [(4,), (5,), (6,), (8,)], '%w%w%w%': [(4, 2, 5)], 'w': [(7,)], 'w%': [(3,), (5,), (7,), (8,), (9,), (13,)]}


100%|██████████| 2528311/2528311 [00:29<00:00, 84820.49it/s]


In [222]:
# queries = ut.read_strings("data/title.title/query/CEB/train.txt")
queries = ut.read_strings("data/link_type.link/query/CEB/train.txt")
df = pd.DataFrame(queries)
# print(df)
df['len'] = df[0].apply(len)
print(df.sort_values('len'))
# print(df)

               0  len
158       spoofs    6
0       % langu%    8
95      %nguage%    8
92      %nced i%    8
90      %nate l%    8
..           ...  ...
155   similar to   10
157   spoofed in   10
159   version of   10
150  featured in   11
151  followed by   11

[160 rows x 2 columns]


In [210]:
for column_name in column_names:
    print(column_name)
    valid_path = f'data/{column_name}/query/CEB/valid.txt'
    train_path = f'data/{column_name}/query/CEB/train.txt'
    train_queries = ut.read_strings(train_path)
    valid_queries = ut.read_strings(valid_path)
    train_queries = sorted(train_queries)
    valid_queries = sorted(valid_queries)
    
    with open(train_path, 'w') as f:
        f.writelines('\n'.join(train_queries))
        
    with open(valid_path, 'w') as f:
        f.writelines('\n'.join(valid_queries))

link_type.link
movie_companies.note
name.name
title.title


In [137]:
db[:10]

['Short, Too',
 'Abdul-Hamid, Jaffar',
 'Al-Hamid, Jaffar Abd',
 "Viera, Michael 'Power'",
 'Buguelo',
 'Seigal, Jason',
 'Starks, Johnny',
 'Monkey',
 "'Morità'",
 "Mark'Oh"]

In [ ]:
ut.data_gen_alg_regex()

In [235]:
queries = ut.read_strings("data/title.title/query/CEB/train.txt")
df = pd.DataFrame(queries)
df['len'] = df[0].apply(len)
display(df)

,0,len
0,!Ne%,4
1,!Next?,6
2,!Que ve el%,11
3,"""A Corrida%",11
4,"""Bookstore"" (%",14
...,...,...
3273221,ìMaten a ese%,13
3273222,"íslenska, on %",14
3273223,île de Sym%,11
3273224,ö,1


In [236]:
len(queries), len(set(queries))

(3273226, 3273226)

In [231]:
wc_dict = {}
for query in queries:
    if query not in wc_dict:
        wc_dict[query] = 0
    wc_dict[query] += 1

In [232]:
for q, c in wc_dict.items():
    if c > 1:
        print(q, c)

100% 2
110% 2
200% 2
Innocent % 2
The 99% 2
The Top 1% 2
